In [ ]:
import os
import rasterio as rio
import rioxarray as rxr
import xarray as xr
import numpy as np
from rasterio.transform import rowcol

os.getcwd()

In [ ]:
raster_path = '' # Add path to imagery here

In [ ]:
# Read in PS image and check number of bands
ps_bands = rio.open(raster_path)
ps_stack = ps_bands.read()
print(ps_stack.shape, ps_stack.dtype, type(ps_stack))
print(ps_bands.crs)

In [ ]:
# Open image
image = rxr.open_rasterio(raster_path, masked=True)

In [ ]:
# # Clip image (optional)

# transform = ps_bands.transform
# width = ps_bands.width
# height = ps_bands.height

# xmin, ymin = 355937, 3936331
# xmax, ymax = 363259, 3940707

# # Top-left pixel
# row_start, col_start = rowcol(transform, xmin, ymax)

# # Bottom-right pixel
# row_stop, col_stop = rowcol(transform, xmax, ymin)

# image = image[:, row_start:row_stop, col_start:col_stop]

In [ ]:
# # Plot clipped image

# import matplotlib.pyplot as plt
# import earthpy.plot as ep

# f, ax = plt.subplots(figsize=(15,10))

# ep.plot_rgb(image.values,
#             rgb=[3, 2, 1],
#             ax=ax,
#             title="False color image",
#             ) 

# plt.show()

In [ ]:
# # Histogram to check band values (~11 min)

# import matplotlib.pyplot as plt
# import matplotlib.ticker as ticker

# fig, axes = plt.subplots(ncols=4, nrows=1, figsize=(20,5))
# ax = axes.ravel()

# blue = image[0]
# green = image[1]
# red = image[2]
# nir = image[3]

# ax[0].hist(blue.values, bins=10)
# ax[0].set_title('Blue Histogram')

# ax[1].hist(green.values, bins=10)
# ax[1].set_title('Green Histogram')

# ax[2].hist(red.values, bins=10)
# ax[2].set_title('Red Histogram')

# ax[3].hist(nir.values, bins=10)
# ax[3].set_title('NIR Histogram')

In [ ]:
# Adding Indices

# NDWI = (GREEN - NIR) / (GREEN + NIR)        
ndwi = (image[1] - image[3]) / (image[1] + image[3])

# NDVI = (NIR - RED) / (NIR + RED)
ndvi = (image[3] - image[2]) / (image[3] + image[2])

# NISI = ( (BLUE + GREEN + RED) - NIR ) / ( (BLUE + GREEN + RED) + NIR )
nisi = ((image[0] + image[1] + image[2]) - image[3]) / ((image[0] + image[1] + image[2]) + image[3])

In [ ]:
# Stack indices onto bands
index_stack = xr.concat([ndwi, ndvi, nisi], dim='band')

PS_stack = xr.concat([
    image[0:4],
    index_stack.assign_coords({'band':['ndwi', 'ndvi', 'nisi']})
],
dim='band')

PS_stack

In [ ]:
PS_stack.attrs['long_name'] = ('blue', 'green', 'red', 'nir', 'ndwi', 'ndvi', 'nisi')
PS_stack

In [ ]:
# # Histogram to check indices values

# import matplotlib.pyplot as plt
# import matplotlib.ticker as ticker

# fig, axes = plt.subplots(ncols=3, nrows=1, figsize=(14,5))
# ax = axes.ravel()

# blue = PS_stack[0]
# green = PS_stack[1]
# red = PS_stack[2]
# nir = PS_stack[3]
# ndwi = PS_stack[4]
# ndvi = PS_stack[5]
# nisi = PS_stack[6]

# ax[0].hist(ndwi.values, bins=10)
# ax[0].set_title('NDWI Histogram')

# ax[1].hist(ndvi.values, bins=10)
# ax[1].set_title('NDVI Histogram')

# ax[2].hist(nisi.values, bins=10)
# ax[2].set_title('NISI Histogram')

# plt.show()

In [ ]:
PS_stack.astype('float32').rio.to_raster('', driver='GTiff') # Add path to export imagery here